## Scott 10K Allied health dataset

### First order of business: create the .txt file of all ADNI T1 patients I want in my table

In [1]:
%%bash 

mydir=/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/
output_file=/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt
>"$output_file"
find "$mydir" -type f -name "*.nii.gz" >> "$output_file"

wc -l<"${output_file}"

15733


In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

t1_paths = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt'
patient_id_list = []
rid_list = []
data_key_list = []
t1_year_list = []
t1_mon_list = []
t1_day_list = []
visdate_list = []

with open(t1_paths,'r') as paths:
    all_paths = paths.readlines()
    for path in all_paths:
        path_parts = path.split('/')
        patient = path_parts[-1].strip('.nii.gz\n')

        patient_parts = patient.split('_')
        data_key = '_'.join(patient_parts)
        data_key_list.append(data_key)
    
        patient_id = '_'.join(patient_parts[:3])
        rid = patient_id.split('_')[-1]
        rid_list.append(rid)
        patient_id_list.append(patient_id)
    
        try:
            t1_date = patient_parts[4]
            t1_year, t1_mon, t1_day = t1_date.split('-')
            t1_year_list.append(t1_year)
            t1_mon_list.append(t1_mon)
            t1_day_list.append(t1_day)
            visdate_list.append(f"{t1_year}-{t1_mon}-{t1_day}")
        except (IndexError, ValueError):
            t1_year_list.append(np.nan)
            t1_mon_list.append(np.nan)
            t1_day_list.append(np.nan)
            visdate_list.append(np.nan)
        
df = {
    "PTID" : patient_id_list,
    "RID" : rid_list,
    "DATA_KEY" : data_key_list,
    "T1_YEAR" : t1_year_list,
    "T1_MON" : t1_mon_list,
    "T1_DAY" : t1_day_list,
    "VISDATE_STR" : visdate_list
}
df = pd.DataFrame(df)

df['T1_YEAR'] = pd.to_numeric(df['T1_YEAR'], errors='coerce')
df['T1_MON'] = pd.to_numeric(df['T1_MON'], errors='coerce')
df['T1_DAY'] = pd.to_numeric(df['T1_DAY'], errors='coerce')

df['VISDATE'] = pd.to_datetime(df['VISDATE_STR'], errors='coerce')
df = df.drop(columns=['VISDATE_STR'])

df['EXAMDATE_4WKS_LATER'] = df['VISDATE'] + timedelta(weeks = 4)
df['EXAMDATE_4WKS_B4'] = df['VISDATE'] - timedelta(weeks = 4)

df      

,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,VISDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4
0,012_S_4849,4849,012_S_4849_ADNI-T1_2012-11-05_12_04_44.0,2012.0,11.0,5.0,2012-11-05,2012-12-03,2012-10-08
1,012_S_4849,4849,012_S_4849_ADNI-T1_2012-07-24_09_50_58.0,2012.0,7.0,24.0,2012-07-24,2012-08-21,2012-06-26
2,012_S_4849,4849,012_S_4849_ADNI-T1_2013-08-26_14_02_58.0,2013.0,8.0,26.0,2013-08-26,2013-09-23,2013-07-29
3,012_S_4849,4849,012_S_4849_ADNI-T1_2013-02-25_13_24_41.0,2013.0,2.0,25.0,2013-02-25,2013-03-25,2013-01-28
4,941_S_6471,6471,941_S_6471_ADNI-T1_2020-09-01_12_46_08.0,2020.0,9.0,1.0,2020-09-01,2020-09-29,2020-08-04
...,...,...,...,...,...,...,...,...,...
15728,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_21_45.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03
15729,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_25_18.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11
15730,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_29_57.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03
15731,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_33_35.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11


### Time to add T1 paths

In [2]:
t1_paths = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt'
key_to_path = {}

with open(t1_paths,'r') as paths:
    all_paths = paths.readlines()
    for path in all_paths:
        path  = path.strip('\n')
        raw_key = path.split('/')[-1]
        key = raw_key.replace('.nii.gz','')
        key_to_path[key] = path

df['T1_PATH'] = df['DATA_KEY'].map(key_to_path)
df

,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,VISDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4,T1_PATH
0,012_S_4849,4849,012_S_4849_ADNI-T1_2012-11-05_12_04_44.0,2012.0,11.0,5.0,2012-11-05,2012-12-03,2012-10-08,/rds/general/project/scott_data_adni/live/ADNI...
1,012_S_4849,4849,012_S_4849_ADNI-T1_2012-07-24_09_50_58.0,2012.0,7.0,24.0,2012-07-24,2012-08-21,2012-06-26,/rds/general/project/scott_data_adni/live/ADNI...
2,012_S_4849,4849,012_S_4849_ADNI-T1_2013-08-26_14_02_58.0,2013.0,8.0,26.0,2013-08-26,2013-09-23,2013-07-29,/rds/general/project/scott_data_adni/live/ADNI...
3,012_S_4849,4849,012_S_4849_ADNI-T1_2013-02-25_13_24_41.0,2013.0,2.0,25.0,2013-02-25,2013-03-25,2013-01-28,/rds/general/project/scott_data_adni/live/ADNI...
4,941_S_6471,6471,941_S_6471_ADNI-T1_2020-09-01_12_46_08.0,2020.0,9.0,1.0,2020-09-01,2020-09-29,2020-08-04,/rds/general/project/scott_data_adni/live/ADNI...
...,...,...,...,...,...,...,...,...,...,...
15728,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_21_45.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03,/rds/general/project/scott_data_adni/live/ADNI...
15729,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_25_18.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11,/rds/general/project/scott_data_adni/live/ADNI...
15730,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_29_57.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03,/rds/general/project/scott_data_adni/live/ADNI...
15731,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_33_35.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11,/rds/general/project/scott_data_adni/live/ADNI...


### Now for mwc1t1 paths

In [3]:
%%bash

mydir=/rds/general/project/c3nl_scott_students/live/data/sankeith/scott_10k_b2c/
output_file=/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/successful_scott_10k_mwc1t1_paths.txt

### collect mwc1t1 file paths##

>"$output_file"
find "$mydir" -type f -name "mwc1t1*" >> "$output_file"
wc -l<"${output_file}"

15726


In [4]:
mwc1t1_paths = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/successful_scott_10k_mwc1t1_paths.txt'
key_to_path = {}

with open(mwc1t1_paths,'r') as paths:
    all_paths = paths.readlines()
    for path in all_paths:
        path  = path.strip('\n')
        key = path.split('/')[-2]
        key_to_path[key] = path

df['MWC1T1_PATH'] = df['DATA_KEY'].map(key_to_path)
df

,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,VISDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4,T1_PATH,MWC1T1_PATH
0,012_S_4849,4849,012_S_4849_ADNI-T1_2012-11-05_12_04_44.0,2012.0,11.0,5.0,2012-11-05,2012-12-03,2012-10-08,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
1,012_S_4849,4849,012_S_4849_ADNI-T1_2012-07-24_09_50_58.0,2012.0,7.0,24.0,2012-07-24,2012-08-21,2012-06-26,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
2,012_S_4849,4849,012_S_4849_ADNI-T1_2013-08-26_14_02_58.0,2013.0,8.0,26.0,2013-08-26,2013-09-23,2013-07-29,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
3,012_S_4849,4849,012_S_4849_ADNI-T1_2013-02-25_13_24_41.0,2013.0,2.0,25.0,2013-02-25,2013-03-25,2013-01-28,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
4,941_S_6471,6471,941_S_6471_ADNI-T1_2020-09-01_12_46_08.0,2020.0,9.0,1.0,2020-09-01,2020-09-29,2020-08-04,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
...,...,...,...,...,...,...,...,...,...,...,...
15728,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_21_45.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
15729,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_25_18.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
15730,073_S_0312,0312,073_S_0312_ADNI-T1_2006-05-31_13_29_57.0,2006.0,5.0,31.0,2006-05-31,2006-06-28,2006-05-03,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
15731,073_S_0312,0312,073_S_0312_ADNI-T1_2007-02-08_16_33_35.0,2007.0,2.0,8.0,2007-02-08,2007-03-08,2007-01-11,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...


#### How many unique patients are there (as measured by PTID)

In [5]:
len(df['PTID'].unique())

2635

In [6]:
df.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/interim_scott_10k_alliedhealth.csv', index = False)

# BREAK - STOP RUNNING HERE #

### Obtain:

#### * ADNI Phase
#### * Site ID
#### * Viscode
#### * Viscode 2
#### * Diagnosis (and other auxillary diagnosis information that starts with DX)

### Merge exactly on EXAMDATE

In [88]:
import pandas as pd

## load up our df and make sure examdate is a datetime object
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/interim_scott_10k_alliedhealth.csv', low_memory = False)
df['EXAMDATE'] = pd.to_datetime(df['EXAMDATE'])

## prep diag: drop relevant columns, get relevant columns as datetime formats, and rename examdate column for diag as that helps with merging
diag = pd.read_csv('/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv', low_memory=False)
diag['EXAMDATE'] = pd.to_datetime(diag['EXAMDATE'])
diag.drop(columns = ['ID','RID','USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR', 'update_stamp'], inplace = True)
print(diag.columns.tolist())
print(len(diag['PTID'].unique()))
print(diag.head())

['PHASE', 'PTID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'DIAGNOSIS', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPROB', 'DXAPOSS', 'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXDEP', 'DXOTHDEM', 'DXODES', 'DXCONFID', 'SITEID']
3787
   PHASE        PTID VISCODE VISCODE2   EXAMDATE  DIAGNOSIS  DXNORM  DXNODEP  \
0  ADNI1  011_S_0002      bl       bl 2005-09-29        1.0     1.0     -4.0   
1  ADNI1  011_S_0003      bl       bl 2005-09-30        3.0    -4.0     -4.0   
2  ADNI1  011_S_0005      bl       bl 2005-09-30        1.0     1.0     -4.0   
3  ADNI1  011_S_0008      bl       bl 2005-09-30        1.0     1.0     -4.0   
4  ADNI1  022_S_0007      bl       bl 2005-10-06        3.0    -4.0     -4.0   

   DXMCI DXMDES  ...  DXAPOSS  DXPARK  DXPDES  DXPCOG  DXPATYP  DXDEP  \
0   -4.0     -4  ...       -4    -4.0    -4.0    -4.0     -4.0    NaN   
1   -4.0     -4  ... 

In [89]:
df_diag_merged = df.merge(diag, on = ['PTID','EXAMDATE'], how = 'left')
df_diag_merged.columns

Index(['PTID', 'RID', 'DATA_KEY', 'T1_YEAR', 'T1_MON', 'T1_DAY', 'EXAMDATE',
       'EXAMDATE_4WKS_LATER', 'EXAMDATE_4WKS_B4', 'T1_PATH', 'MWC1T1_PATH',
       'PHASE', 'VISCODE', 'VISCODE2', 'DIAGNOSIS', 'DXNORM', 'DXNODEP',
       'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4',
       'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD',
       'DXAPP', 'DXAPROB', 'DXAPOSS', 'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP',
       'DXDEP', 'DXOTHDEM', 'DXODES', 'DXCONFID', 'SITEID'],
      dtype='object')

In [90]:
df_diag_merged.shape

(15737, 42)

In [91]:
t1s = df_diag_merged["T1_PATH"]
df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")

/var/tmp/pbs.1012943.pbs-7/ipykernel_935837/3759790812.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")


,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,EXAMDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4,T1_PATH,MWC1T1_PATH
14867,009_S_1030,1030,009_S_1030_ADNI-T1_2010-12-08_14_08_53.0,2010.0,12.0,8.0,2010-12-08,2011-01-05,2010-11-10,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
14868,009_S_1030,1030,009_S_1030_ADNI-T1_2014-12-01_13_43_42.0,2014.0,12.0,1.0,2014-12-01,2014-12-29,2014-11-03,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
898,123_S_0108,0108,123_S_0108_ADNI-T1_2010-02-25_09_32_24.0,2010.0,2.0,25.0,2010-02-25,2010-03-25,2010-01-28,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
897,123_S_0108,0108,123_S_0108_ADNI-T1_2011-02-15_08_58_12.0,2011.0,2.0,15.0,2011-02-15,2011-03-15,2011-01-18,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
8595,127_S_0431,0431,127_S_0431_ADNI-T1_2007-01-04_12_46_32.0,2007.0,1.0,4.0,2007-01-04,2007-02-01,2006-12-07,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
8596,127_S_0431,0431,127_S_0431_ADNI-T1_2007-06-28_15_24_54.0,2007.0,6.0,28.0,2007-06-28,2007-07-26,2007-05-31,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
10119,168_S_6634,6634,168_S_6634_ADNI-T1_2021-02-16_13_16_14.0,2021.0,2.0,16.0,2021-02-16,2021-03-16,2021-01-19,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...
10118,168_S_6634,6634,168_S_6634_ADNI-T1_2023-07-05_09_18_12.0,2023.0,7.0,5.0,2023-07-05,2023-08-02,2023-06-07,/rds/general/project/scott_data_adni/live/ADNI...,/rds/general/project/c3nl_scott_students/live/...


In [92]:
## had to do some bullshit trudging through the ADNI database: all but 073_S_7052's true visit codes have been ID'ed. We'll keep 073_S_0752's data in for now but
## I'm dropping the others by index
## For row 14867, I had to drop it after finding this patient's screening MMSE was taken on the 9th of March, but the T1 scan I have is from the 17th, so our T1 scan is likely for baseline.
df_diag_merged.drop(index=[8595, 10119, 898, 14867], inplace=True)
t1s = df_diag_merged["T1_PATH"]
df_diag_merged[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")

,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,EXAMDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4,T1_PATH,...,DXAPOSS,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXODES,DXCONFID,SITEID


In [93]:
print(f' number of scans with matching diagnoses = {df_diag_merged["DIAGNOSIS"].notna().sum()}')

 number of scans with matching diagnoses = 2563


In [94]:
for index,row in df['T1_PATH'].items():
    if os.path.exists(row) == False:
        print(row)

In [95]:
df_diag_merged.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/2_interim_scott_10k_alliedhealth.csv', index = False)

### Obtain demographics data (merge demographics data within the bounds of 4 weeks)
### BUT, I want to calculate the approximate age of a patient and time of their MRI scan via EXAMDATE - (approximate) DATE OF BIRTH

In [96]:
## Load up demographics df
demog_df = pd.read_csv('/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/PTDEMOG_10Sep2025.csv',low_memory=False)
print(f' Number of unique patients in the ADNI demographics CSV = {len(demog_df['PTID'].unique())}')

## Drop unwanted columns, and rename VISDATE so I can use it for merging with my original dataframe
demog_df.drop(axis = 1,columns = ['PHASE','RID','VISCODE', 'VISCODE2', 'SITEID', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1', 'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2', 'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3', 'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4', 'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5', 'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6', 'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN', 'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR', 'ID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR', 'update_stamp'], inplace = True)
demog_df.rename(columns = {'VISDATE':'EXAMDATE'}, inplace = True)
demog_df['EXAMDATE'] = pd.to_datetime(demog_df['EXAMDATE'])

## Construct a column called 'APPROX_DOB': it's like PTDOBm but adds the day as the first of that month (PTDOB is MM/YYYY but APPROX_DOB is 01/MM/YYYY)
demog_df['PTDOB'] = demog_df['PTDOB'].astype(str)
demog_df['APPROX_DOB'] = pd.to_datetime("01/" + demog_df['PTDOB'], errors='coerce', format = "%d/%m/%Y")
print(demog_df.columns.tolist())
print(demog_df.shape)

 Number of unique patients in the ADNI demographics CSV = 4942
['PTID', 'EXAMDATE', 'PTSOURCE', 'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM', 'PTSPOTTIM', 'APPROX_DOB']
(6199, 33)


In [97]:
## prep df
df= pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/2_interim_scott_10k_alliedhealth.csv')
df['EXAMDATE'] = pd.to_datetime(df['EXAMDATE'])
print(f' Number of unique patients in the Greg n Laura CSV = {len(df['PTID'].unique())}')
print(df.columns.tolist())
print(df.shape)

 Number of unique patients in the Greg n Laura CSV = 2635
['PTID', 'RID', 'DATA_KEY', 'T1_YEAR', 'T1_MON', 'T1_DAY', 'EXAMDATE', 'EXAMDATE_4WKS_LATER', 'EXAMDATE_4WKS_B4', 'T1_PATH', 'MWC1T1_PATH', 'PHASE', 'VISCODE', 'VISCODE2', 'DIAGNOSIS', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPROB', 'DXAPOSS', 'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXDEP', 'DXOTHDEM', 'DXODES', 'DXCONFID', 'SITEID']
(15733, 42)


In [98]:
from dateutil.relativedelta import relativedelta

# Rename EXAMDATE in demog_df
demog_df.rename(columns={'EXAMDATE': 'EXAMDATE_DEMOGRAPHICS'}, inplace=True)

# Merge on PTID
merged_df = df.merge(demog_df, on='PTID', how='left', suffixes=('', '_demog'))

# Filter for the date window
mask = (
    (merged_df['EXAMDATE_DEMOGRAPHICS'] >= merged_df['EXAMDATE_4WKS_B4']) & 
    (merged_df['EXAMDATE_DEMOGRAPHICS'] <= merged_df['EXAMDATE_4WKS_LATER'])
)

# Calculate time difference for valid matches (in days)
merged_df.loc[mask, 'TIME_DIFF_DEMOGRAPHICS'] = abs(
    merged_df.loc[mask, 'EXAMDATE'] - merged_df.loc[mask, 'EXAMDATE_DEMOGRAPHICS']
).dt.days

# For each original row, keep only the closest demographic match
merged_df = merged_df.loc[mask].sort_values('TIME_DIFF_DEMOGRAPHICS').drop_duplicates(
    subset=df.columns.tolist(), keep='first'
)

# Merge back with original to preserve unmatched rows
final_df = df.merge(
    merged_df.drop(columns=df.columns.tolist()),
    left_index=True, right_index=True, how='left'
)

# NEW: Propagate APPROX_DOB and PTDOB across all scans for each patient
# For each PTID, fill in APPROX_DOB and PTDOB from any matched scan
for col in ['APPROX_DOB', 'PTDOB', 'PTGENDER', 'PTETHCAT', 'PTRACCAT']:
    if col in final_df.columns:
        final_df[col] = final_df.groupby('PTID')[col].transform(lambda x: x.ffill().bfill().infer_objects(copy = False))
        pd.set_option('future.no_silent_downcasting', True)

# Calculate APPROX_AGE using relativedelta (vectorized approach)
final_df['APPROX_AGE'] = final_df.apply(
    lambda row: relativedelta(row['EXAMDATE'], row['APPROX_DOB']).years 
    if pd.notna(row['APPROX_DOB']) and pd.notna(row['EXAMDATE']) else None, 
    axis=1
)

# Check for duplicates in T1_PATH
t1s = final_df["T1_PATH"]
print(final_df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH"))
print(f"\nFinal merged dataframe shape: {final_df.shape}")
print(f"Number of rows with demographics matched: {final_df['EXAMDATE_DEMOGRAPHICS'].notna().sum()}")
print(f"Number of rows with APPROX_DOB (after propagation): {final_df['APPROX_DOB'].notna().sum()}")
print(f"Number of rows with APPROX_AGE calculated: {final_df['APPROX_AGE'].notna().sum()}")
print(f"\ntime_diff statistics:\n{final_df['TIME_DIFF_DEMOGRAPHICS'].describe()}")
print(f"\nAPPROX_AGE statistics:\n{final_df['APPROX_AGE'].describe()}")

Empty DataFrame
Columns: [PTID, RID, DATA_KEY, T1_YEAR, T1_MON, T1_DAY, EXAMDATE, EXAMDATE_4WKS_LATER, EXAMDATE_4WKS_B4, T1_PATH, MWC1T1_PATH, PHASE, VISCODE, VISCODE2, DIAGNOSIS, DXNORM, DXNODEP, DXMCI, DXMDES, DXMPTR1, DXMPTR2, DXMPTR3, DXMPTR4, DXMPTR5, DXMPTR6, DXMDUE, DXMOTHET, DXDSEV, DXDDUE, DXAD, DXAPP, DXAPROB, DXAPOSS, DXPARK, DXPDES, DXPCOG, DXPATYP, DXDEP, DXOTHDEM, DXODES, DXCONFID, SITEID, EXAMDATE_DEMOGRAPHICS, PTSOURCE, PTGENDER, PTDOB, PTDOBYY, PTHAND, PTMARRY, PTEDUCAT, PTWORKHS, PTWORK, PTNOTRT, PTRTYR, PTHOME, PTTLANG, PTPLANG, PTADBEG, PTCOGBEG, PTADDX, PTETHCAT, PTRACCAT, PTIDENT, PTORIENT, PTORIENTOT, PTENGSPK, PTNLANG, PTENGSPKAGE, PTCLANG, PTLANGSP, PTLANGWR, PTSPTIM, PTSPOTTIM, APPROX_DOB, TIME_DIFF_DEMOGRAPHICS, APPROX_AGE]
Index: []

[0 rows x 76 columns]

Final merged dataframe shape: (15733, 76)
Number of rows with demographics matched: 2081
Number of rows with APPROX_DOB (after propagation): 10215
Number of rows with APPROX_AGE calculated: 10210

time_dif

In [99]:
final_df.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/3_interim_scott_10k_alliedhealth.csv', index = False)

### Merge MMSE (within the bounds of 4 weeks!)

In [100]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np # For NaN
from dateutil.relativedelta import relativedelta

### Load df and make EXAMDATE a datetime object
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/3_interim_scott_10k_alliedhealth.csv', low_memory = False)
df['EXAMDATE'] = pd.to_datetime(df['EXAMDATE'], errors='coerce')

### Load MMSE df, create datetime object from EXAMDATE_MMSE
mmse_df = pd.read_csv('/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MMSE_28Mar2025.csv', low_memory=False)
mmse_df.rename(columns = {'VISDATE':'EXAMDATE_MMSE'}, inplace = True)
mmse_df['EXAMDATE_MMSE'] = pd.to_datetime(mmse_df['EXAMDATE_MMSE'], errors='coerce')
mmse_df.drop(columns = ['PHASE', 'RID', 'VISCODE', 'VISCODE2','ID', 'SITEID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR', 'update_stamp'], inplace = True)

print(f' Number of unique patients in MMSE df: {len(mmse_df["PTID"].unique())}')
print(mmse_df.columns.tolist())

# Merge on PTID
merged_df = df.merge(mmse_df, on='PTID', how='left', suffixes=('', '_mmse'))

# Filter for the date window
mask = (
    (merged_df['EXAMDATE_MMSE'] >= merged_df['EXAMDATE_4WKS_B4']) & 
    (merged_df['EXAMDATE_MMSE'] <= merged_df['EXAMDATE_4WKS_LATER'])
)

# Calculate time difference for valid matches (in days)
merged_df.loc[mask, 'TIME_DIFF_MMSE'] = abs(
    merged_df.loc[mask, 'EXAMDATE'] - merged_df.loc[mask, 'EXAMDATE_MMSE']
).dt.days

# For each original row, keep only the closest MMSE match
merged_df = merged_df.loc[mask].sort_values('TIME_DIFF_MMSE').drop_duplicates(
    subset=df.columns.tolist(), keep='first'
)

# Merge back with original to preserve unmatched rows
final_df = df.merge(
    merged_df.drop(columns=df.columns.tolist()),
    left_index=True, right_index=True, how='left'
)

# Check for duplicates in T1_PATH
t1s = final_df["T1_PATH"]
duplicates = final_df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")
if len(duplicates) > 0:
    print("Duplicate T1_PATH entries found:")
    print(duplicates)
else:
    print("No duplicate T1_PATH entries found")

print(f"Final merged dataframe shape: {final_df.shape}")
print(f"Number of rows with MMSE matched: {final_df['EXAMDATE_MMSE'].notna().sum()}")
print(f"TIME_DIFF_MMSE statistics:\n{final_df['TIME_DIFF_MMSE'].describe()}")

 Number of unique patients in MMSE df: 4475
['PTID', 'EXAMDATE_MMSE', 'DONE', 'NDREASON', 'SOURCE', 'MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE', 'WORDLIST', 'WORD1', 'WORD2', 'WORD3', 'MMTRIALS', 'MMD', 'MML', 'MMR', 'MMO', 'MMW', 'MMLTR1', 'MMLTR2', 'MMLTR3', 'MMLTR4', 'MMLTR5', 'MMLTR6', 'MMLTR7', 'WORLDSCORE', 'WORD1DL', 'WORD2DL', 'WORD3DL', 'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR', 'MMREAD', 'MMWRITE', 'MMDRAW', 'MMSCORE']
No duplicate T1_PATH entries found
Final merged dataframe shape: (15733, 122)
Number of rows with MMSE matched: 1973
TIME_DIFF_MMSE statistics:
count    1973.000000
mean        5.637101
std         7.383791
min         0.000000
25%         0.000000
50%         1.000000
75%        10.000000
max        28.000000
Name: TIME_DIFF_MMSE, dtype: float64


In [101]:
mmse_df.shape

(14119, 46)

In [102]:
final_df.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/4_interim_scott_10k_alliedhealth.csv', index = False)

### Merge APOE (visit agnostic)

In [3]:
import pandas as pd
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/4_interim_scott_10k_alliedhealth.csv', low_memory = False)
apoe_df = pd.read_csv('/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/APOERES_28Mar2025.csv', low_memory=False)
final_apoe_df = apoe_df[['PTID','GENOTYPE']].drop_duplicates(subset = 'PTID', keep = 'first')

final_df = df.merge(final_apoe_df, on = 'PTID', how = 'left')

t1s = final_df["T1_PATH"]
duplicates = final_df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")
print(duplicates)

print(f"Final merged dataframe shape: {final_df.shape}")
print(f"Number of rows with APOE genotype: {final_df['GENOTYPE'].notna().sum()}")

final_df.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/5_interim_scott_10k_alliedhealth.csv', index = False)

Empty DataFrame
Columns: [PTID, RID, DATA_KEY, T1_YEAR, T1_MON, T1_DAY, EXAMDATE, EXAMDATE_4WKS_LATER, EXAMDATE_4WKS_B4, T1_PATH, MWC1T1_PATH, PHASE, VISCODE, VISCODE2, DIAGNOSIS, DXNORM, DXNODEP, DXMCI, DXMDES, DXMPTR1, DXMPTR2, DXMPTR3, DXMPTR4, DXMPTR5, DXMPTR6, DXMDUE, DXMOTHET, DXDSEV, DXDDUE, DXAD, DXAPP, DXAPROB, DXAPOSS, DXPARK, DXPDES, DXPCOG, DXPATYP, DXDEP, DXOTHDEM, DXODES, DXCONFID, SITEID, EXAMDATE_DEMOGRAPHICS, PTSOURCE, PTGENDER, PTDOB, PTDOBYY, PTHAND, PTMARRY, PTEDUCAT, PTWORKHS, PTWORK, PTNOTRT, PTRTYR, PTHOME, PTTLANG, PTPLANG, PTADBEG, PTCOGBEG, PTADDX, PTETHCAT, PTRACCAT, PTIDENT, PTORIENT, PTORIENTOT, PTENGSPK, PTNLANG, PTENGSPKAGE, PTCLANG, PTLANGSP, PTLANGWR, PTSPTIM, PTSPOTTIM, APPROX_DOB, TIME_DIFF_DEMOGRAPHICS, APPROX_AGE, EXAMDATE_MMSE, DONE, NDREASON, SOURCE, MMDATE, MMYEAR, MMMONTH, MMDAY, MMSEASON, MMHOSPIT, MMFLOOR, MMCITY, MMAREA, MMSTATE, WORDLIST, WORD1, WORD2, WORD3, MMTRIALS, MMD, MML, MMR, MMO, MMW, ...]
Index: []

[0 rows x 123 columns]
Final mer

In [4]:
apoe_ptids = apoe_df["PTID"]
duplicates = apoe_df[apoe_ptids.isin(apoe_ptids[apoe_ptids.duplicated()])].sort_values("PTID")
print(duplicates)

Empty DataFrame
Columns: [PHASE, PTID, RID, VISCODE, GENOTYPE, APTESTDT, APVOLUME, APRECEIVE, APAMBTEMP, APRESAMP, APUSABLE, ID, SITEID, USERDATE, USERDATE2, update_stamp]
Index: []


In [104]:
###Greg demanded checks for rows with a bland filename

null_mask = final_df['MWC1T1_PATH'].isna()
null_rows = final_df[null_mask]
null_rows

,PTID,RID,DATA_KEY,T1_YEAR,T1_MON,T1_DAY,EXAMDATE,EXAMDATE_4WKS_LATER,EXAMDATE_4WKS_B4,T1_PATH,...,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,MMSCORE,TIME_DIFF_MMSE,GENOTYPE
4135,024_S_1393,1393,024_S_1393_ADNI-T1_2008-04-07_10_59_15.0,2008.0,4.0,7.0,2008-04-07,2008-05-05,2008-03-10,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/3
10305,016_S_4952,4952,016_S_4952_ADNI-T1_2017-02-14_07_22_09.0,2017.0,2.0,14.0,2017-02-14,2017-03-14,2017-01-17,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/4
11269,003_S_0981,0981,003_S_0981_ADNI-T1_2009-11-11_13_50_14.0,2009.0,11.0,11.0,2009-11-11,2009-12-09,2009-10-14,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/3
11403,136_S_0086,0086,136_S_0086_ADNI-T1_2009-04-28_13_41_59.0,2009.0,4.0,28.0,2009-04-28,2009-05-26,2009-03-31,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/3
13154,027_S_0120,0120,027_S_0120_ADNI-T1_2006-08-16_13_32_10.0,2006.0,8.0,16.0,2006-08-16,2006-09-13,2006-07-19,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/4
13706,136_S_0874,0874,136_S_0874_ADNI-T1_2009-05-05_10_34_10.0,2009.0,5.0,5.0,2009-05-05,2009-06-02,2009-04-07,/rds/general/project/scott_data_adni/live/ADNI...,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,30.0,1.0,4/4
14541,002_S_0938,0938,002_S_0938_ADNI-T1_2007-10-23_09_35_43.0,2007.0,10.0,23.0,2007-10-23,2007-11-20,2007-09-25,/rds/general/project/scott_data_adni/live/ADNI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/3


### Merging LIMMTOTAL (bounds of 4 weeks)

In [105]:
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/5_interim_scott_10k_alliedhealth.csv', low_memory = False)
df['EXAMDATE'] = pd.to_datetime(df['EXAMDATE'])

nbat_df = pd.read_csv('/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/NEUROBAT_28Mar2025.csv', low_memory = False)
nbat_df = nbat_df[['PTID','VISDATE','LIMMTOTAL','AVDELTOT','AVTOT6','AVTOTB']]
nbat_df['VISDATE'] = pd.to_datetime(nbat_df['VISDATE'])

nbat_df.rename(columns = {'VISDATE':'EXAMDATE_NBAT'}, inplace = True)

print(f' Number of unique patients in NBAT df: {len(nbat_df["PTID"].unique())}')
print(nbat_df.columns.tolist())

# Merge on PTID
merged_df = df.merge(nbat_df, on='PTID', how='left', suffixes=('', '_nbat'))

# Filter for the date window - CORRECTED: use EXAMDATE_MMSE instead of EXAMDATE_DEMOGRAPHICS
mask = (
    (merged_df['EXAMDATE_NBAT'] >= merged_df['EXAMDATE_4WKS_B4']) & 
    (merged_df['EXAMDATE_NBAT'] <= merged_df['EXAMDATE_4WKS_LATER'])
)

# Calculate time difference for valid matches (in days)
merged_df.loc[mask, 'TIME_DIFF_NBAT'] = abs(
    merged_df.loc[mask, 'EXAMDATE'] - merged_df.loc[mask, 'EXAMDATE_NBAT']
).dt.days

# For each original row, keep only the closest MMSE match
merged_df = merged_df.loc[mask].sort_values('TIME_DIFF_NBAT').drop_duplicates(
    subset=df.columns.tolist(), keep='first'
)

# Merge back with original to preserve unmatched rows
final_df = df.merge(
    merged_df.drop(columns=df.columns.tolist()),
    left_index=True, right_index=True, how='left'
)

# Check for duplicates in T1_PATH
t1s = final_df["T1_PATH"]
duplicates = final_df[t1s.isin(t1s[t1s.duplicated()])].sort_values("T1_PATH")
if len(duplicates) > 0:
    print("Duplicate T1_PATH entries found:")
    print(duplicates)
else:
    print("No duplicate T1_PATH entries found")

print(f"Final merged dataframe shape: {final_df.shape}")
print(f"Number of rows with NBAT data matched: {final_df['EXAMDATE_NBAT'].notna().sum()}")
print(f"TIME_DIFF_NBAT statistics:\n{final_df['TIME_DIFF_NBAT'].describe()}")

 Number of unique patients in NBAT df: 4512
['PTID', 'EXAMDATE_NBAT', 'LIMMTOTAL', 'AVDELTOT', 'AVTOT6', 'AVTOTB']
No duplicate T1_PATH entries found
Final merged dataframe shape: (15733, 129)
Number of rows with NBAT data matched: 1784
TIME_DIFF_NBAT statistics:
count    1784.000000
mean        5.361547
std         6.800243
min         0.000000
25%         0.000000
50%         1.000000
75%         9.000000
max        28.000000
Name: TIME_DIFF_NBAT, dtype: float64


In [106]:
final_df.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/6_interim_scott_10k_alliedhealth.csv',index = False)